# Detecting change in Australian forestry

**Notebook currently compatible with the `DEA Sandbox` environment (untested on `NCI`)**

### General advice (delete this cell before submitting for review)

- When choosing a location for your analysis, **select an area that has data on both the `NCI` and `DEA Sandbox`** to allow your code to be run on both environments. 
For example, you can check this for Landsat using the [DEA Explorer](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3/1990) (use the drop-down menu to view all products). 
As of September 2019, the `DEA Sandbox` has a single year of continental Landsat data for 2015-16, and the full 1987-onward time-series for three locations (Perth WA, Brisbane QLD, and western NSW).
- When writing in Markdown cells, start each sentence is on a **new line**.
This makes it easy to see changes through git commits.
- Use Australian English in markdown cells and code comments.
- Use the [PEP8 standard](https://www.python.org/dev/peps/pep-0008/) for code. To make sure all code in the notebook is consistent, you can use the `jupyterlab_code_formatter` tool: select each code cell, then click `Edit` and then one of the `Apply X Formatter` options (`YAPF` or `Black` are recommended). This will reformat the code in the cell to a consistent style.
- In the final notebook cell, include a set of relevant tags which are used to build the DEA User Guide's [Tag Index](https://docs.dea.ga.gov.au/genindex.html). 
Use all lower-case, seperate words with spaces, and where possible re-use existing tags.
Ensure the tags cell below is in `Raw` format, rather than `Markdown` or `Code`.


### Background
An *optional* overview of the scientific, economic or environmental management issue or challenge being addressed by Digital Earth Australia. 
For `Beginners_Guide` or `Frequently_Used_Code` notebooks, this may include information about why the particular technique or approach is useful or required. 
If you need to cite a scientific paper or link to a website, use a persistent DOI link if possible and link in-text (e.g. [Dhu et al. 2017](https://doi.org/10.1080/20964471.2017.1402490)).

### The problem
A _compulsory_ description of the notebook, including a brief overview of how Digital Earth Australia helps to address the problem set out above, and a run-down of the tools/methods being demonstrated below.


### Digital Earth Australia use case

### Technical details
* **Products used:** [`s2_ard_granule`](https://explorer.sandbox.dea.ga.gov.au/s2a_ard_granule), [`s2b_ard_granule`](https://explorer.sandbox.dea.ga.gov.au/s2b_ard_granule)
* **Analyses used:** NDVI index, image differencing, hypothesis testing

## Getting started
**To run this analysis**, run all the cells in the notebook, starting with the "Load packages" cell.

**When you finished the analysis**, you can return to the "Analysis parameters" cell, modify some values (e.g. choose a different location or time period to analyse) and re-run the analysis.
There are additional instructions on modifying the notebook at the end.

### Load packages

In [ ]:
%matplotlib inline

import datacube
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import sys
import xarray as xr

sys.path.append("../Scripts")
from dea_datahandling import load_ard
from dea_bandindices import calculate_indices
from dea_plotting import display_map, rgb

### Connect to the datacube
Give your datacube app a unique name that is consistent with the purpose of the notebook.

In [ ]:
dc = datacube.Datacube(app="Change_detection")

### Analysis parameters

An *optional* section to inform the user of any parameters they'll need to configure to run the notebook:
* `param_name_1`: Simple description (e.g. `example_value`). Advice about appropriate values to choose for this parameter.
* `param_name_2`: Simple description (e.g. `example_value`). Advice about appropriate values to choose for this parameter.


In [ ]:
latitude = (-35.271, -35.331)
longitude = (149.256, 149.356)

time = ('2019-06-01', '2019-09-01')
time_baseline = np.datetime64('2019-07-15')

## View the selected location
The next cell will display the selected area on an interactive map.
Feel free to zoom in and out to get a better understanding of the area you'll be analysing.
Clicking on any point of the map will reveal the latitude and longitude coordinates of that point.

In [ ]:
display_map(x=longitude, y=latitude)

## Load and view Sentinel 2 data

In [ ]:
products = ['s2a_nrt_granule', 's2b_nrt_granule']

query = {
    "x": longitude,
    "y": latitude,
    "time": time,
    "measurements": [
        "nbart_red",
        "nbart_green",
        "nbart_blue",
        "nbart_nir_1",
        "fmask"
    ],
    "output_crs": "EPSG:3577",
    "resolution": (-10, 10)
}

ds_s2 = load_ard(dc, products=products, min_gooddata=0.7, **query)

### Plot example timestep in true colour

In [ ]:
rgb(ds_s2, index=0)

## Compute band indices
We'll measure vegetation with the normalised difference vegetation index (NDVI), which we can calculate using the in-built `calculate_indices` function.

In [ ]:
# Use code comments for low-level documentation of code
ds_s2 = calculate_indices(ds_s2, 'NDVI', collection='ga_sentinel2_1')

In [ ]:
fig = plt.figure(figsize=(10.5,7))
ds_s2.NDVI.isel(time=0).plot(cmap='RdYlGn')
plt.show()

## Perform hypothesis test
We want to test whether there has been significant change in the mean NDVI before and after our baseline date.

### Make samples

In [ ]:
baseline_sample = ds_s2.NDVI.sel(time=ds_s2['time']<=time_baseline)
postbaseline_sample = ds_s2.NDVI.sel(time=ds_s2['time']>time_baseline)

# Record coodrinates for reconstructing xarray objects
sample_lat_coords = ds_s2.coords['y']
sample_lon_coords = ds_s2.coords['x']

In [ ]:
print(sample_lat_coords)

### Test for change
Perform a two-sided t-test to check if the mean NDVI has changed significantly after the baseline.

In [ ]:
tstat, p_tstat = stats.ttest_ind(
    postbaseline_sample.values,
    baseline_sample.values,
    equal_var=False,
    nan_policy='omit',
)

# Convert results to an xarray
t_test = xr.Dataset(
    {
        't_stat': (['x', 'y'], tstat),
        'p_val': (['x', 'y'], p_tstat)
    },
    coords={
        'x': (['x'], sample_lat_coords.values),
        'y': (['y'], sample_lon_coords.values)
    })

print(t_test)

## Visualise change
From the test, we're interested in two conditions: whether the change is significant (rejection of our null hypothesis) and whether the change was positive (afforestation) or negative (deforestation).

We can reject the null hypothesis if our $p$-value (`p_val`) is less than our chosen significance level, which we set as `sig_level = 0.05`.

We can infer the direction of the change from the value of `t_stat`, which is propotional to $$\text{mean(post baseline)} - \text{mean(baseline)}.$$
This means that
- if the NDVI is **higher** in the `post baseline` sample compared to the `baseline` sample, then `t_stat` will be **positive**.
- if the NDVI is **lower** in the `post baseline` sample compared to the `baseline` sample, then `t_stat` will be **negative**.

In [ ]:
# Set the significance level
sig_level = 0.05

# Calculate masks to indicate significant positive and negative change
t_test['sig_pos_change'] = (t_test.p_val < sig_level) & (t_test.t_stat > 0)
t_test['sig_neg_change'] = (t_test.p_val < sig_level) & (t_test.t_stat < 0)

# Plot the mask for significant positive change
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
t_test.sig_pos_change.plot()
ax.set_title("Areas of significant afforestation")
plt.show()

# Plot the mask for significant negative change
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
t_test.sig_neg_change.plot()
ax.set_title("Areas of significant deforestation")
plt.show()

# Plot the difference in the means
fig, ax = plt.subplots(1, 1, figsize=(7, 5))
(postbaseline_sample.mean(dim=['time']) - baseline_sample.mean(dim=['time'])).plot()
ax.set_title("Difference in the mean")
plt.show()

### Extract polygons around areas of change

In [ ]:
import rasterio.features

sig_neg_change_poly = rasterio.features.shapes(t_test.sig_neg_change.data.astype('float32'))
sig_neg_change_poly_list = list(sig_neg_change_poly)

In [ ]:
RemoveZero = [i for i, x in enumerate(sig_neg_change_poly_list) if x[1] == 1]
sig_neg_pol_list = [sig_neg_change_poly_list[x] for x in RemoveZero]
print(len(sig_neg_pol_list))

In [ ]:
from shapely.geometry import Polygon, shape

polyshapes = []
for i, polyshape in enumerate(sig_neg_pol_list):
    polyArea = Polygon(polyshape[0]['coordinates'][0]).area
    polyshape[0]['properties'] = {'area': polyArea}
    polyshapes.append(polyshape)

In [ ]:
AreasIndex = [i for i, x in enumerate(polyshapes) if x[0]['properties']['area'] > 1000]
polybig = [polyshapes[x] for x in AreasIndex]
print(len(polybig))

In [ ]:
from descartes import PolygonPatch

polybigbreaktuple = [a for a, b in list(polybig)]

fig = plt.figure(figsize = [7,7])
ax = fig.gca()
for i in range(len(polybig)):
    ax.add_patch(PolygonPatch(polybigbreaktuple[i], edgecolor = 'b', facecolor = None))
ax.axis('scaled')
plt.draw()
plt.show()

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** September 2019

**Compatible `datacube` version:** 

In [ ]:
print(datacube.__version__)

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)